<a href="https://colab.research.google.com/github/xmks-colab/sfalsin/blob/main/__karaoke__.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#@title Carregar arquivos{ display-mode: "form" }
from google.colab import files

uploaded = files.upload()

# Se quiser ver os arquivos enviados:
print("\n📂 Arquivos enviados:")
for filename in uploaded.keys():
    print(filename)

Saving letra.txt to letra.txt

📂 Arquivos enviados:
letra.txt


In [5]:
#@title Gerenciado de arquivos simples{ display-mode: "form" }
import os
import shutil
import ipywidgets as widgets
from IPython.display import display, clear_output, Javascript
import threading

class FileManager:
    print("Apagar,Mover,Copiar caminho")
    def __init__(self, start_path='/content'):
        self.current_path = start_path
        self.output = widgets.Output()
        self.path_label = widgets.Label()
        self.status_label = widgets.Label()
        self.item_to_move = None
        self.refresh()

    def list_directory(self, path):
        try:
            items = os.listdir(path)
            items.sort()
            return items
        except Exception as e:
            return [f"Error: {e}"]

    def change_directory(self, folder):
        new_path = os.path.join(self.current_path, folder)
        if os.path.isdir(new_path):
            self.current_path = new_path
            self.refresh()

    def go_up(self):
        parent = os.path.dirname(self.current_path)
        if os.path.exists(parent):
            self.current_path = parent
            self.refresh()

    def create_folder(self):
        folder_name_input = widgets.Text(
            placeholder='Digite o nome da pasta',
            description='Nome:',
            style={'description_width': 'initial'}
        )
        create_btn = widgets.Button(description='✅ Criar', button_style='success')
        cancel_btn = widgets.Button(description='❌ Cancelar')

        def do_create(btn):
            folder_name = folder_name_input.value.strip()
            if folder_name:
                new_folder_path = os.path.join(self.current_path, folder_name)
                try:
                    os.makedirs(new_folder_path, exist_ok=False)
                    self.set_status(f"✅ Pasta '{folder_name}' criada com sucesso!")
                    self.refresh()
                except FileExistsError:
                    self.set_status(f"❌ Pasta '{folder_name}' já existe!")
                except Exception as e:
                    self.set_status(f"❌ Erro ao criar pasta: {e}")
            else:
                self.set_status("❌ Nome da pasta não pode estar vazio!")

        def cancel_create(btn):
            self.refresh()

        with self.output:
            clear_output()
            self.path_label.value = f"📁 {self.current_path}"
            display(widgets.VBox([
                self.path_label,
                self.status_label,
                widgets.HTML("<h4>📁 Criar Nova Pasta</h4>"),
                folder_name_input,
                widgets.HBox([create_btn, cancel_btn])
            ]))

        create_btn.on_click(do_create)
        cancel_btn.on_click(cancel_create)

    def delete_item(self, item):
        full_path = os.path.join(self.current_path, item)
        confirm = widgets.Button(description=f'❗ Confirmar apagar: {item}', button_style='danger')
        cancel = widgets.Button(description='Cancelar')

        def do_delete(btn):
            try:
                if os.path.isdir(full_path):
                    shutil.rmtree(full_path)
                else:
                    os.remove(full_path)
                self.set_status(f"✅ {item} apagado com sucesso!")
                self.refresh()
            except Exception as e:
                self.set_status(f"❌ Erro ao apagar {item}: {e}")

        def cancel_delete(btn):
            self.refresh()

        with self.output:
            clear_output()
            self.path_label.value = f"📁 {self.current_path}"
            display(widgets.VBox([
                self.path_label,
                self.status_label,
                widgets.HTML(f"<h4>🗑️ Confirmar Exclusão</h4>"),
                widgets.HTML(f"<p>Tem certeza que deseja apagar: <strong>{item}</strong>?</p>"),
                widgets.HBox([confirm, cancel])
            ]))

        confirm.on_click(do_delete)
        cancel.on_click(cancel_delete)

    def mark_for_move(self, item):
        self.item_to_move = os.path.join(self.current_path, item)
        self.set_status(f"📦 Item selecionado para mover: {os.path.basename(item)}")
        self.refresh()

    def paste_item(self, btn=None):
        if self.item_to_move:
            src = self.item_to_move
            dst = os.path.join(self.current_path, os.path.basename(src))
            try:
                self.set_status("⏳ Movendo...")
                shutil.move(src, dst)
                self.item_to_move = None
                self.set_status("✅ Movido com sucesso!")
            except Exception as e:
                self.set_status(f"❌ Erro ao mover: {e}")
            self.refresh()

    def set_status(self, msg, duration=3):
        self.status_label.value = msg
        if duration > 0:
            threading.Timer(duration, self.clear_status).start()

    def clear_status(self):
        self.status_label.value = ""

    def refresh(self):
        with self.output:
            clear_output()
            self.path_label.value = f"📁 {self.current_path}"


            display(widgets.VBox([
                self.path_label,
                self.status_label
            ]))

            rows = []

            # Botão para voltar
            if self.current_path != '/':
                up_button = widgets.Button(description='🔼 ..', layout=widgets.Layout(width='auto'))
                up_button.on_click(lambda btn: self.go_up())
                rows.append(widgets.HBox([up_button]))

            # Botão para criar pasta
            create_folder_btn = widgets.Button(description='📁➕ Nova Pasta', button_style='info', layout=widgets.Layout(width='auto'))
            create_folder_btn.on_click(lambda btn: self.create_folder())
            rows.append(widgets.HBox([create_folder_btn]))

            # Botão para colar item (se houver)
            if self.item_to_move:
                paste_btn = widgets.Button(description=f'📥 Colar aqui: {os.path.basename(self.item_to_move)}', button_style='success')
                paste_btn.on_click(self.paste_item)
                rows.append(widgets.HBox([paste_btn]))

            # Lista de itens do diretório
            items = self.list_directory(self.current_path)
            for item in items:
                full_path = os.path.join(self.current_path, item)
                label = f"📁 {item}" if os.path.isdir(full_path) else f"📄 {item}"

                open_btn = widgets.Button(description=label, layout=widgets.Layout(width='55%'))
                del_btn = widgets.Button(description='️A', layout=widgets.Layout(width='10%'), button_style='danger')
                move_btn = widgets.Button(description='M', layout=widgets.Layout(width='10%'))

                copy_btn = widgets.Button(description='C', layout=widgets.Layout(width='10%'), button_style='primary')
                def make_copy_callback(path=full_path):
                    def cb(btn):
                        js_code = f'''
                        navigator.clipboard.writeText("{path}").then(function() {{
                            console.log("Copiado com sucesso!");
                        }}, function(err) {{
                            console.error("Erro ao copiar: ", err);
                        }});
                        '''
                        display(Javascript(js_code))
                        self.set_status(f"📋 Copiado: {os.path.basename(path)}")
                    return cb
                copy_btn.on_click(make_copy_callback())

                if os.path.isdir(full_path):
                    open_btn.on_click(lambda btn, folder=item: self.change_directory(folder))

                del_btn.on_click(lambda btn, name=item: self.delete_item(name))
                move_btn.on_click(lambda btn, name=item: self.mark_for_move(name))

                rows.append(widgets.HBox([open_btn, del_btn, move_btn, copy_btn]))

            display(widgets.VBox(rows))

    def show(self):
        display(self.output)

# Exemplo de uso:
fm = FileManager()
fm.show()

Apagar,Mover,Copiar caminho


Output()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [49]:

#@title Configurações { display-mode: "form" }
import ipywidgets as widgets
from IPython.display import display, clear_output

# Widgets

txt_path_widget = widgets.Text(
    value='/content/letra.txt',
    description='TXT:',
    layout=widgets.Layout(width='500px')
)

maiusculas_widget = widgets.Checkbox(value=True, description='Converter para maiúsculas')

largura_widget = widgets.IntText(value=1280, description='Largura:')
altura_widget = widgets.IntText(value=720, description='Altura:')
tamanho_fonte_widget = widgets.IntSlider(value=48, min=10, max=150, step=2, description='Fonte:')
usar_fonte_widget = widgets.Checkbox(value=True, description='Usar fonte personalizada')
fonte_personalizada_widget = widgets.Text(
    value='/content/minhaFonte.ttf',
    description='Fonte TTF:',
    layout=widgets.Layout(width='500px')
)

linhas_por_imagem_widget = widgets.IntSlider(value=4, min=1, max=20, description='Linhas:')
espaco_entre_linhas_widget = widgets.IntSlider(value=20, min=0, max=100, description='Espaço :')
usar_borda_widget = widgets.Checkbox(value=True, description='Usar borda?')
espessura_borda_widget = widgets.IntSlider(value=2, min=1, max=10, description='Espessura borda:')

cor_texto_widget = widgets.ColorPicker(value='#FFFFFF', description='Cor texto:')
cor_borda_widget = widgets.ColorPicker(value='#000000', description='Cor borda:')

modo_linha_individual_widget = widgets.Checkbox(value=True, description='Modo linha individual?')

# Botão
button = widgets.Button(description='Aplicar Configurações')

output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output()
        # Converte cores HEX para RGBA
        def hex_to_rgba(hex_color):
            h = hex_color.lstrip('#')
            return tuple(int(h[i:i+2], 16) for i in (0, 2, 4)) + (255,)

        # Variáveis finais
        global txt_path, maiusculas, largura, altura, tamanho_fonte, usar_fonte_personalizada
        global fonte_personalizada, linhas_por_imagem, espaco_entre_linhas
        global usar_borda, cor_texto, cor_borda, espessura_borda, modo_linha_individual

        txt_path = txt_path_widget.value
        maiusculas = maiusculas_widget.value
        largura = largura_widget.value
        altura = altura_widget.value
        tamanho_fonte = tamanho_fonte_widget.value
        usar_fonte_personalizada = usar_fonte_widget.value
        fonte_personalizada = fonte_personalizada_widget.value
        linhas_por_imagem = linhas_por_imagem_widget.value
        espaco_entre_linhas = espaco_entre_linhas_widget.value
        usar_borda = usar_borda_widget.value
        espessura_borda = espessura_borda_widget.value
        cor_texto = hex_to_rgba(cor_texto_widget.value)
        cor_borda = hex_to_rgba(cor_borda_widget.value)
        modo_linha_individual = modo_linha_individual_widget.value

        print('✅ Configurações aplicadas com sucesso!')
        print('\nSe quiser ver os valores gerados, execute:')
        print('\nprint(txt_path, maiusculas, largura, altura, tamanho_fonte, usar_fonte_personalizada, fonte_personalizada, linhas_por_imagem, espaco_entre_linhas, usar_borda, cor_texto, cor_borda, espessura_borda, modo_linha_individual)')

button.on_click(on_button_clicked)

# Layout da interface
display(
    txt_path_widget, maiusculas_widget, largura_widget, altura_widget, tamanho_fonte_widget,
    usar_fonte_widget, fonte_personalizada_widget,
    linhas_por_imagem_widget, espaco_entre_linhas_widget,
    usar_borda_widget, espessura_borda_widget,
    cor_texto_widget, cor_borda_widget,
    modo_linha_individual_widget,
    button, output
)

Text(value='/content/letra.txt', description='TXT:', layout=Layout(width='500px'))

Checkbox(value=True, description='Converter para maiúsculas')

IntText(value=1280, description='Largura:')

IntText(value=720, description='Altura:')

IntSlider(value=48, description='Fonte:', max=150, min=10, step=2)

...

ColorPicker(value='#FFFFFF', description='Cor texto:')

ColorPicker(value='#000000', description='Cor borda:')

Checkbox(value=True, description='Modo linha individual?')

Button(description='Aplicar Configurações', style=ButtonStyle())

Output()

In [47]:

#@title Executar { display-mode: "form" }
from PIL import Image, ImageDraw, ImageFont
import os


output_folder = '/content/imagens/'
os.makedirs(output_folder, exist_ok=True)

# === FUNÇÕES ===

def carregar_fonte():
    if usar_fonte_personalizada and os.path.exists(fonte_personalizada):
        try:
            print(f'Usando fonte personalizada: {fonte_personalizada}')
            return ImageFont.truetype(fonte_personalizada, tamanho_fonte)
        except Exception as e:
            print(f'Erro ao carregar fonte personalizada: {e}')
    print('Tentando carregar Arial...')
    try:
        return ImageFont.truetype("arial.ttf", tamanho_fonte)
    except:
        print('Arial não encontrada, usando fonte padrão do PIL.')
        return ImageFont.load_default()

def get_text_size(text, font, draw):
    bbox = draw.textbbox((0, 0), text, font=font)
    return bbox[2] - bbox[0], bbox[3] - bbox[1]

def desenhar_texto_com_borda(draw, x, y, texto, fonte):
    # Desenha contorno
    if usar_borda:
        for dx in range(-espessura_borda, espessura_borda+1):
            for dy in range(-espessura_borda, espessura_borda+1):
                if dx!=0 or dy!=0:
                    draw.text((x+dx, y+dy), texto, font=fonte, fill=cor_borda)
    # Texto
    draw.text((x, y), texto, font=fonte, fill=cor_texto)

def criar_imagem_bloco(linhas, ys, font, index):
    img = Image.new('RGBA', (largura, altura), (255,255,255,0))
    draw = ImageDraw.Draw(img)
    for linha, y in zip(linhas, ys):
        w, _ = get_text_size(linha, font, draw)
        x = (largura - w)//2
        desenhar_texto_com_borda(draw, x, y, linha, font)
    path = f'{output_folder}imagem_{index:03}.png'
    img.save(path)
    print(f'Imagem bloco salva: {path}')

def criar_imagens_linha_a_linha(linhas, ys, font, base_index):
    for i, (linha, y) in enumerate(zip(linhas, ys), start=1):
        img = Image.new('RGBA', (largura, altura), (255,255,255,0))
        draw = ImageDraw.Draw(img)
        w, _ = get_text_size(linha, font, draw)
        x = (largura - w)//2
        desenhar_texto_com_borda(draw, x, y, linha, font)
        path = f'{output_folder}imagem_{base_index:03}_linha{i}.png'
        img.save(path)
        print(f'Imagem linha salva: {path}')

# === EXECUÇÃO ===

font = carregar_fonte()

with open(txt_path, 'r', encoding='utf-8') as f:
    todas_linhas = [l.strip() for l in f if l.strip()]

# Converte para maiúsculas se a opção estiver ativada
if maiusculas:
    todas_linhas = [linha.upper() for linha in todas_linhas]

contador = 1
# Processa em blocos de N linhas
for i in range(0, len(todas_linhas), linhas_por_imagem):
    bloco = todas_linhas[i:i+linhas_por_imagem]
    # Calcula posições Y
    dummy_img = Image.new('RGBA',(1,1),0); draw = ImageDraw.Draw(dummy_img)
    heights = [get_text_size(l, font, draw)[1] for l in bloco]
    total_h = sum(heights) + (len(heights)-1)*espaco_entre_linhas
    start_y = (altura - total_h)//2
    ys = []
    y = start_y
    for h in heights:
        ys.append(y)
        y += h + espaco_entre_linhas
    # Gera imagens
    if modo_linha_individual:
        criar_imagens_linha_a_linha(bloco, ys, font, contador)
    else:
        criar_imagem_bloco(bloco, ys, font, contador)
    contador += 1

print(f'\n✅ Todas as imagens foram salvas em: {output_folder}')

Usando fonte personalizada: /content/alte_din.ttf
Imagem bloco salva: /content/imagens/imagem_001.png
Imagem bloco salva: /content/imagens/imagem_002.png

✅ Todas as imagens foram salvas em: /content/imagens/


In [48]:
#@title Baixar imagens{ display-mode: "form" }
import shutil
import os
from google.colab import files
import glob


for zip_file in glob.glob('/content/*.zip'):
    try:
        os.remove(zip_file)
        print(f'🗑️ ZIP apagado: {zip_file}')
    except Exception as e:
        print(f'Erro ao apagar {zip_file}: {e}')


folder_path = '/content/imagens'  # <- Troque aqui para o nome da sua pasta


zip_name = 'arquivo.zip'
zip_path = f'/content/{zip_name}'


shutil.make_archive(zip_path.replace('.zip', ''), 'zip', folder_path)
print(f'\n✅ Novo ZIP criado: {zip_path}')

# 5. Iniciar o download
print('\n⬇️ Iniciando o download...')
files.download(zip_path)


✅ Novo ZIP criado: /content/arquivo.zip

⬇️ Iniciando o download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [45]:
 #@title Apagar pastas das imagens(recomendo a cada execução) { display-mode: "form" }
import shutil
import os

def apagar_pasta(caminho_da_pasta):
    try:
        # Verifica se o caminho existe
        if os.path.exists(caminho_da_pasta):
            # Remove a pasta e todo seu conteúdo
            shutil.rmtree(caminho_da_pasta)
            print(f"Pasta '{caminho_da_pasta}' apagada com sucesso!")
        else:
            print(f"Pasta '{caminho_da_pasta}' não encontrada.")
    except Exception as e:
        print(f"Erro ao apagar a pasta: {e}")

# Exemplo de uso:
apagar_pasta('/content/imagens')

Pasta '/content/imagens' apagada com sucesso!
